In [20]:
# Suprimir advertencias
import warnings
warnings.filterwarnings("ignore")


#Importing Libraries:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
import datetime
from sklearn.feature_selection import chi2
from scipy.stats import chi2_contingency

In [21]:
#Loading Data:
data = pd.read_csv("IFPTML-OD+MCSD.csv", sep=';')
data.head(10)

,Type of data.,f(vij)obs,prediction_trainig,fref,sIcI,sIcIV,sIIcIV,sIIIcII,sIVcII,sIVcIII,sVcIV,sVIcI,sVIIcII
0,Original,0,training,0.4167,0.171,0.737,1.102,0.000,0.000,0.000,0.967,0.000,0.593
1,Original,1,training,0.4167,0.010,1.240,0.000,0.000,1.200,1.533,1.615,0.000,0.000
2,Original,1,training,0.4167,0.244,1.647,0.677,0.632,0.000,0.000,1.164,0.000,0.000
3,Original,1,validation,0.4167,0.306,1.190,0.723,0.818,0.788,1.034,1.153,0.000,0.887
4,Original,1,training,0.4167,0.069,0.521,1.004,0.000,0.348,1.094,0.471,0.128,0.478
5,Original,0,training,0.4167,0.244,0.606,0.328,0.221,0.084,0.857,0.164,0.139,0.162
6,Original,1,training,0.4167,0.592,1.302,0.677,0.000,0.000,0.000,1.027,0.540,0.077
7,Original,0,validation,0.4167,0.250,1.045,0.451,0.000,0.005,0.242,0.306,0.000,0.175
8,Original,1,training,0.4167,0.011,0.720,0.000,0.048,0.000,0.000,0.832,0.000,0.284
9,Original,1,training,0.4167,0.450,0.519,0.315,0.169,0.036,0.032,0.368,0.581,0.144


In [22]:
#Data Splitting:
Y = data.iloc[:, 1]
X = data.iloc[:, 3:]

In [23]:
#Splitting Data into Training and Prediction Sets:
train_data = data[data["prediction_trainig"] == "training"]
pred_data = data[data["prediction_trainig"] == "validation"]

In [24]:
# Model Preparation for LDA
X_train_LDA= train_data.iloc[:, 3:]
X_test_LDA= pred_data.iloc[:, 3:]
y_train_LDA= train_data.iloc[:, 1]
y_test_LDA= pred_data.iloc[:, 1]

print(X_train_LDA.shape)
print(X_test_LDA.shape)
print(y_train_LDA.shape)
print(y_test_LDA.shape)

(1744, 10)
(524, 10)
(1744,)
(524,)


In [25]:
#X_LDA
mlr_X_train_LDA= X_train_LDA[['fref', 'sIcI', 'sIcIV','sIIcIV','sIIIcII', 'sIVcII', 'sIVcIII','sVcIV','sVIcI','sVIIcII']]
mlr_X_test_LDA= X_test_LDA[['fref', 'sIcI', 'sIcIV','sIIcIV','sIIIcII', 'sIVcII', 'sIVcIII','sVcIV','sVIcI','sVIIcII']]

In [26]:
#Model_LDA

In [27]:
# Definir las probabilidades a priori de clasificación
priors = [0.5, 0.5]  

In [28]:
# Inicializar el modelo LDA con las probabilidades a priori
LDA = LinearDiscriminantAnalysis(priors=priors)

In [29]:
LDA.fit(mlr_X_train_LDA, y_train_LDA)

LinearDiscriminantAnalysis(priors=[0.5, 0.5])

In [30]:
y_pred_train_LDA = LDA.predict(mlr_X_train_LDA)
y_pred_test_LDA = LDA.predict(mlr_X_test_LDA)

In [31]:
# Realizar validación cruzada con Linear Discriminant Analysis
cv_scores = cross_val_score(LDA, mlr_X_train_LDA, y_train_LDA, cv=5)  # 5-fold cross-validation
print("Cross-validation Scores:", cv_scores)

Cross-validation Scores: [0.78510029 0.75931232 0.78510029 0.79083095 0.86206897]


In [42]:
# Get the names of the variables
variable_names = X.columns

In [33]:
# Coeficientes de la función discriminante lineal
coefficients = LDA.coef_[0]
intercept = LDA.intercept_[0]

In [43]:
# Print the equation of the line with variable names
print("Equation of the line:")
print("y = ", end="")
for i, coefficient in enumerate(coefficients):
    variable_name = variable_names[i]
    print(f"({coefficient:.2f} * {variable_name}) + ", end="")
print(f"{intercept:.2f}")

Equation of the line:
y = (11.78 * fref) + (-0.57 * sIcI) + (2.54 * sIcIV) + (-1.13 * sIIcIV) + (0.52 * sIIIcII) + (-0.03 * sIVcII) + (-0.32 * sIVcIII) + (1.18 * sVcIV) + (0.13 * sVIcI) + (-0.89 * sVIIcII) + -4.49


In [35]:
confusion_train = confusion_matrix(y_train_LDA,y_pred_train_LDA)
confusion_test = confusion_matrix(y_test_LDA, y_pred_test_LDA)

In [36]:
y_train_pred_proba = LDA.predict_proba(mlr_X_train_LDA)
y_test_pred_proba = LDA.predict_proba(mlr_X_test_LDA)

In [37]:
print('Confusion Matrix for Training Series')
print(confusion_train)
print('Confusion Matrix for Test Series')
print(confusion_test)

Confusion Matrix for Training Series
[[1120  268]
 [  73  283]]
Confusion Matrix for Test Series
[[343 105]
 [ 12  64]]


In [38]:
tn, fp, fn, tp = confusion_matrix(y_train_LDA, y_pred_train_LDA).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
accuracy = (tn+tp)/(tn+tp+fn+fp)

In [39]:
print('Sp(%)=', specificity,'Sn(%)=' , sensitivity,'Ac(%)=' , accuracy)

Sp(%)= 0.8069164265129684 Sn(%)= 0.7949438202247191 Ac(%)= 0.8044724770642202


In [40]:
tn, fp, fn, tp = confusion_matrix(y_test_LDA, y_pred_test_LDA).ravel()
specificity = tn / (tn+fp)
sensitivity = tp / (tp+fn)
accuracy = (tn+tp)/(tn+tp+fn+fp)

In [41]:
print('Sp(%)=', specificity,'Sn(%)=' , sensitivity,'Ac(%)=' , accuracy),

Sp(%)= 0.765625 Sn(%)= 0.8421052631578947 Ac(%)= 0.7767175572519084


(None,)